In [ ]:
import os
from tqdm import tqdm
import pandas as pd
import sys
import time
import logging
import numpy as np
import matplotlib.pyplot as plt
import cv2
import math


In [ ]:
import re
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

In [ ]:
video_path = '/Users/andrei-macpro/Documents/Data/videos/1059_meal.mp4'

In [ ]:
openpose_path = '/Users/andrei-macpro/Documents/Data/openpose_output/1059_meal/'

In [ ]:
poses = [x for x in sorted_alphanumeric(os.listdir(openpose_path))]

In [ ]:
poses_frames = [int(x.split('.')[0].split("_")[1]) for x in poses]

In [ ]:
cap = cv2.VideoCapture(video_path)
frames = []
count = 0
while(cap.isOpened()):
        ret_val, image = cap.read()
        if ret_val == False:
            break
        frames.append(image)
        count = count+1

In [ ]:
for i in range(len(frames)):
    if i not in poses_frames and i!=0:
        frames.pop(i-1)


In [ ]:
len(frames)

In [ ]:
coords = pd.read_csv(openpose_path+pose_frames, index_col=0)

In [ ]:
len(poses_frames)

In [ ]:
coords.round(0)

In [ ]:
person_1 = (coords['x_1'].max()-coords['x_1'].min()) * (coords['y_1'].max()-coords['y_1'].min())
person_2 = (coords['x_2'].max()-coords['x_2'].min()) * (coords['y_2'].max()-coords['y_2'].min())
person_3 = (coords['x_3'].max()-coords['x_3'].min()) * (coords['y_3'].max()-coords['y_3'].min())

In [ ]:
person_2


In [ ]:
rect = frames[2][int(coords['y_3'].min()):int(coords['y_3'].max()), int(coords['x_3'].min()):int(coords['x_3'].max())]

In [ ]:
plt.imshow(rect)

In [ ]:
rect = frame[0][int(coords['y_3'].min()):int(coords['y_3'].max()), int(coords['x_3'].min()):int(coords['x_3'].max())]

In [ ]:
## right arm length: 2 to 4
# euclidean distance: sqrt((x2 -x1)*2 + (y2-y1)*2))

person_1 = math.sqrt( (coords.at[1,'x_1'] - coords.at[3,'x_1'])**2 + (coords.at[1,'y_1'] - coords.at[3,'y_1'])**2)

person_2 = math.sqrt( (coords.at[1,'x_2'] - coords.at[3,'x_2'])**2 + (coords.at[1,'y_2'] - coords.at[3,'y_2'])**2)

person_3 = math.sqrt( (coords.at[1,'x_3'] - coords.at[3,'x_3'])**2 + (coords.at[1,'y_3'] - coords.at[3,'y_3'])**2)


In [ ]:
output_path = '/Users/andrei-macpro/Documents/Data/openpose_output/1059_meal_skeleton/'


In [ ]:
os.chdir(output_path)
missing_values = []
count = 0
for pose,frame in zip(poses[::100], frames[::100]):
    coords = pd.read_csv(openpose_path+pose, index_col=0)
    coords = coords.round(0)
    if len(coords.columns)/3== 3:
        # if missing data in right arm use left arm
        if pd.isnull(coords.at[1,'x_1']) == pd.isnull(coords.at[3,'x_1']) == pd.isnull(coords.at[1,'y_1']) == \
        pd.isnull(coords.at[3,'y_1'])== pd.isnull(coords.at[1,'x_2']) == pd.isnull(coords.at[3,'x_2']) == \
        pd.isnull(coords.at[1,'y_2']) == pd.isnull(coords.at[3,'y_2'])== pd.isnull(coords.at[1,'x_3']) == \
        pd.isnull(coords.at[3,'x_3']) == pd.isnull(coords.at[1,'y_3']) == pd.isnull(coords.at[3,'y_3'])== False:
            right_1 = math.sqrt( (coords.at[1,'x_1'] - coords.at[3,'x_1'])**2 + (coords.at[1,'y_1'] - coords.at[3,'y_1'])**2)
            right_2 = math.sqrt( (coords.at[1,'x_2'] - coords.at[3,'x_2'])**2 + (coords.at[1,'y_2'] - coords.at[3,'y_2'])**2)
            right_3 = math.sqrt( (coords.at[1,'x_3'] - coords.at[3,'x_3'])**2 + (coords.at[1,'y_3'] - coords.at[3,'y_3'])**2)
            right_hand = np.array([right_1, right_2, right_3])
            if np.argmin(right_hand) == 2:
                smallest = frame[int(coords['y_3'].min()):int(coords['y_3'].max()), int(coords['x_3'].min()):int(coords['x_3'].max())]
                cv2.imwrite(pose.split('.')[0] + '.png', smallest)
            elif np.argmin(right_hand) == 1:
                smallest = frame[int(coords['y_2'].min()):int(coords['y_2'].max()), int(coords['x_2'].min()):int(coords['x_2'].max())]
                cv2.imwrite(pose.split('.')[0] + '.png', smallest)
            elif np.argmin(right_hand) == 0:
                smallest = frame[int(coords['y_1'].min()):int(coords['y_1'].max()), int(coords['x_1'].min()):int(coords['x_1'].max())]
                cv2.imwrite(pose.split('.')[0]+ '.png', smallest)
                
        elif pd.isnull(coords.at[4,'x_1']) == pd.isnull(coords.at[6,'x_1']) == pd.isnull(coords.at[4,'y_1']) == pd.isnull(coords.at[6,'y_1'])\
        == pd.isnull(coords.at[4,'x_2']) == pd.isnull(coords.at[6,'x_2']) == pd.isnull(coords.at[4,'y_2']) == pd.isnull(coords.at[6,'y_2'])\
        == pd.isnull(coords.at[4,'x_3']) == pd.isnull(coords.at[6,'x_3']) == pd.isnull(coords.at[4,'y_3']) == pd.isnull(coords.at[6,'y_3'])\
        == False:
            left_1 = math.sqrt( (coords.at[4,'x_1'] - coords.at[6,'x_1'])**2 + (coords.at[4,'y_1'] - coords.at[6,'y_1'])**2)
            left_2 = math.sqrt( (coords.at[4,'x_2'] - coords.at[6,'x_2'])**2 + (coords.at[4,'y_2'] - coords.at[6,'y_2'])**2)
            left_3 = math.sqrt( (coords.at[4,'x_3'] - coords.at[6,'x_3'])**2 + (coords.at[4,'y_3'] - coords.at[6,'y_3'])**2)
            left_hand = np.array([left_1, left_2, left_3])
            if np.argmin(left_hand) == 2:
                smallest = frame[int(coords['y_3'].min()):int(coords['y_3'].max()), int(coords['x_3'].min()):int(coords['x_3'].max())]
                cv2.imwrite(pose.split('.')[0]+ '.png', smallest)
            elif np.argmin(right_hand) == 1:
                smallest = frame[int(coords['y_2'].min()):int(coords['y_2'].max()), int(coords['x_2'].min()):int(coords['x_2'].max())]
                cv2.imwrite(pose.split('.')[0]+'.png', smallest)
            elif np.argmin(right_hand) == 0:
                smallest = frame[int(coords['y_1'].min()):int(coords['y_1'].max()), int(coords['x_1'].min()):int(coords['x_1'].max())]
                cv2.imwrite(pose.split('.')[0]+ '.png', smallest)
        else:
            missing_values.append(pose.split('.')[0])
            

                
    

In [ ]:
len(missing_values)

In [ ]:
plt.imshow(frames[0])

In [ ]:
(coords.at[1,'x_3'] - coords.at[3,'x_3'])*2 + (coords.at[1,'y_3'] - coords.at[3,'y_3'])*2

In [ ]:
math.sqrt(-96)

In [ ]:
(coords.at[1,'x_3'] - coords.at[3,'x_3'])*2

In [ ]:
coords.at[1,'x_3'] - coords.at[3,'x_3']

In [ ]:
np.argmin(np.array([person_1, person_3, person_2]))

In [ ]:
type(coords.at[1,'x_1'], coords.at[3,'x_1'], coords.at[2,'x_1'])

In [ ]:
type(coords.round(0).at[1,'x_1'])

In [ ]:
coords.at[6,'x_1'] == 'NaN'

In [ ]:
coords.at[6,'x_1']

In [ ]:
pd.isnull(coords.at[6,'x_1'])

In [ ]:
coords['x_1'].loc[1]